# Manejando Datos Faltantes

La diferencia entre los datos que se encuentran en muchos tutoriales y los datos en el mundo real es que los datos reales rara vez son limpios y homogéneos. En particular, muchos conjuntos de datos interesantes tendrán cierta cantidad de datos faltantes. Para complicar aún más las cosas, diferentes fuentes de datos pueden indicar los datos faltantes de diferentes maneras.

En esta sección, hablaremos sobre algunas consideraciones generales para los datos faltantes, veremos cómo Pandas elige representarlos y exploraremos algunas herramientas incorporadas de Pandas para manejar datos faltantes en Python.

## Compensaciones en las Convenciones de Datos Faltantes

Se han desarrollado varios enfoques para rastrear la presencia de datos faltantes en una tabla o DataFrame. Generalmente, giran en torno a una de dos estrategias: usar una máscara que indique globalmente los valores faltantes, o elegir un valor centinela que indique una entrada faltante.

En el enfoque de enmascaramiento, la máscara podría ser un array booleano completamente separado, o podría implicar la apropiación de un bit en la representación de datos para indicar localmente el estado nulo de un valor.

En el enfoque del centinela, el valor centinela podría ser alguna convención específica de los datos, como indicar un valor entero faltante con –9999 o algún patrón de bits raro, o podría ser una convención más global, como indicar un valor de punto flotante faltante con NaN (Not a Number), un valor especial que forma parte de la especificación de punto flotante IEEE.

Ninguno de estos enfoques está libre de compensaciones. El uso de un array de máscara separado requiere la asignación de un array booleano adicional, lo que añade sobrecarga tanto en almacenamiento como en cálculo. Un valor centinela reduce el rango de valores válidos que pueden ser representados, y puede requerir lógica adicional (a menudo no optimizada) en la aritmética de CPU y GPU, porque valores especiales comunes como NaN no están disponibles para todos los tipos de datos.

Como en la mayoría de los casos donde no existe una elección universalmente óptima, diferentes lenguajes y sistemas utilizan diferentes convenciones. Por ejemplo, el lenguaje R utiliza patrones de bits reservados dentro de cada tipo de datos como valores centinela que indican datos faltantes, mientras que el sistema SciDB utiliza un byte adicional adjunto a cada celda para indicar un estado NA.

## Datos Faltantes en Pandas

La forma en que Pandas maneja los valores faltantes está limitada por su dependencia del paquete NumPy, que no tiene una noción incorporada de valores NA para tipos de datos que no son de punto flotante.

Quizás Pandas podría haber seguido el ejemplo de R al especificar patrones de bits para cada tipo de datos individual para indicar nulidad, pero este enfoque resulta ser bastante difícil de manejar. Mientras que R tiene solo 4 tipos de datos principales, NumPy admite muchos más: por ejemplo, mientras que R tiene un solo tipo de entero, NumPy admite 14 tipos de enteros básicos una vez que se tienen en cuenta los anchos de bits disponibles, la firma y la endianidad de la codificación. Reservar un patrón de bits específico en todos los tipos disponibles de NumPy llevaría a una cantidad excesiva de sobrecarga en casos especiales de varias operaciones para varios tipos, probablemente incluso requiriendo una nueva bifurcación del paquete NumPy. Además, para los tipos de datos más pequeños (como enteros de 8 bits), sacrificar un bit para usarlo como máscara reduciría significativamente el rango de valores que puede representar.

Debido a estas limitaciones y compensaciones, Pandas tiene dos "modos" de almacenar y manipular valores nulos:

1. El modo predeterminado es usar un esquema de datos faltantes basado en centinela, con valores centinela NaN o None dependiendo del tipo de datos.
2. Alternativamente, puedes optar por usar los tipos de datos anulables (dtypes) que proporciona Pandas (discutidos más adelante), lo que resulta en la creación de un array de máscara adjunto para rastrear entradas faltantes. Estas entradas faltantes luego se presentan al usuario como el valor especial pd.NA.

En cualquier caso, las operaciones y manipulaciones de datos proporcionadas por la API de Pandas manejarán y propagarán esas entradas faltantes de manera predecible. Pero para desarrollar cierta intuición sobre por qué se toman estas decisiones, profundicemos rápidamente en las compensaciones inherentes a None, NaN y NA. Como de costumbre, comenzaremos importando NumPy y Pandas.

In [ ]:
import numpy as np
import pandas as pd

## None como Valor Centinela

In [ ]:
vals1 =  np.array([1, None, 2, 3])
vals1

array([1, None, 2, 3], dtype=object)

Este `dtype=object` significa que la mejor representación de tipo común que NumPy pudo inferir para el contenido del array es que son objetos de Python. La desventaja de usar `None` de esta manera es que las operaciones con los datos se realizarán a nivel de Python, con mucha más sobrecarga que las operaciones rápidas que suelen observarse en arrays con tipos nativos:

In [ ]:
%timeit np.arange(1_000_000, dtype=int).sum()

908 µs ± 25.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%timeit np.arange(1_000_000, dtype=object).sum()

70.4 ms ± 3.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

## NaN

El estándar IEEE para punto flotante (específicamente IEEE 754) es una norma técnica que define cómo los números decimales deben ser representados y procesados en sistemas informáticos.

En este estándar, "NaN" (Not a Number) es un valor especial reservado para representar resultados indefinidos o errores en operaciones matemáticas. Algunas características importantes de NaN:

Es parte del estándar IEEE 754 para representación de punto flotante, adoptado por prácticamente todos los procesadores y lenguajes de programación modernos.

NaN es el resultado de operaciones matemáticamente indefinidas como 0/0 o la raíz cuadrada de un número negativo.

A diferencia de otros valores especiales, NaN tiene la propiedad única de que no es igual a nada, ni siquiera a sí mismo (es decir, NaN != NaN es verdadero).

Las operaciones aritméticas que involucran un NaN generalmente producen otro NaN como resultado.

NaN solo existe para tipos de datos de punto flotante (como float o double), no para enteros u otros tipos de datos.

Esta estandarización de NaN permite a lenguajes como Python/Pandas tener una forma consistente de representar valores faltantes o indefinidos en datos numéricos de punto flotante, que es reconocida universalmente por el hardware y software que implementa el estándar IEEE 754.

In [ ]:
vals2 = np.array([1, np.nan, 3, 4])
vals2

array([ 1., nan,  3.,  4.])

In [ ]:
vals2.sum(), vals2.min(), vals2.max()

(np.float64(nan), np.float64(nan), np.float64(nan))

In [ ]:
1 + np.nan

nan

In [ ]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(np.float64(8.0), np.float64(1.0), np.float64(4.0))

## NaN y None en Pandas

In [ ]:
pd.Series([1, np.nan, 2, None])

,0
0,1.0
1,NaN
2,2.0
3,NaN


In [ ]:
x = pd.Series(range(2), dtype=int)
x

,0
0,0
1,1


In [ ]:
x[0] = None
x

,0
0,NaN
1,1.0


|Typeclass     | Conversion when storing NAs | NA sentinel value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

## Pandas Nullables Dtypes
En las primeras versiones de Pandas, `NaN` y `None` como valores centinela eran las únicas representaciones de datos faltantes disponibles. La principal dificultad que esto introdujo se relacionaba con la conversión implícita de tipos: por ejemplo, no era posible representar una matriz de enteros verdadera con datos faltantes.

Para solucionar esta dificultad, Pandas añadió posteriormente *tipos de datos nulos*, que se distinguen de los tipos de datos normales por el uso de mayúsculas en sus nombres (p. ej., `pd.Int32` frente a `np.int32`). Para compatibilidad con versiones anteriores, estos tipos de datos nulos solo se utilizan si se solicita específicamente.

Por ejemplo, a continuación se muestra una `Serie` de enteros con datos faltantes, creada a partir de una lista que contiene los tres marcadores de datos faltantes disponibles:

In [ ]:
pd.Series([1, np.nan, 2, None, pd.NA], dtype='Int32')

,0
0,1
1,<NA>
2,2
3,<NA>
4,<NA>


In [ ]:
# Datos  de ventas mensuales con valores faltantes
ventas = [10, 15, None, 20, np.nan, 30]

serie_ventas = pd.Series(ventas)

print(serie_ventas.dtype)
print(serie_ventas)

float64
0    10.0
1    15.0
2     NaN
3    20.0
4     NaN
5    30.0
dtype: float64


In [ ]:
ventas = [10, 15, None, 20, np.nan, 30]

serie_ventas_int = pd.Series(ventas, dtype='Int64') # I con mayúscula

print(serie_ventas_int.dtype)
print(serie_ventas_int)

Int64
0      10
1      15
2    <NA>
3      20
4    <NA>
5      30
dtype: Int64


### Ventajas
1. **Mantiene la integridad del tipo**: Los datos se mantienen como enteros, no se convierten a flotantes.
2. **Representación coherente**: Todos los valores faltantes se representan uniformemente como `<NA>`.
3. **Operaciones matemáticas consistentes**: Las operaciones manejan los valores faltantes de manera predecible.
4. **Claridad semántica**: Es más claro que estamos trabajando con enteros que accidentalmente tienen valores faltantes, no con números que potencialmente tienen decimales.

## Operando con Datos Null

In [ ]:
data = pd.Series([1, np.nan, 'hello', None])

In [ ]:
data.isnull()

,0
0,False
1,True
2,False
3,True


In [ ]:
data[data.notnull()]

,0
0,1
2,hello


In [ ]:
data.dropna()

,0
0,1
2,hello


In [ ]:
df = pd.DataFrame([[1,      np.nan, 2],
                  [2,      3,      5],
                  [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [ ]:
df.dropna()

,0,1,2
1,2.0,3.0,5


In [ ]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


In [ ]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [ ]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [ ]:
df.dropna(axis='rows', thresh=3) # El valor mínimo de valores no nulos que tiene que tener una fila para no ser borrada

,0,1,2,3
1,2.0,3.0,5,NaN


In [ ]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'), dtype='Int32')
data

,0
a,1
b,<NA>
c,2
d,<NA>
e,3


In [ ]:
data.fillna(0)

,0
a,1
b,0
c,2
d,0
e,3


In [ ]:
data.ffill()

,0
a,1
b,1
c,2
d,2
e,3


Este método propaga el último valor conocido hacia adelante hasta encontrar el siguiente valor no-nulo. Es útil para series temporales donde puedes asumir que el valor permanece constante hasta la siguiente medición. Por ejemplo, en precios de acciones, datos de sensores o cualquier medición donde "sin cambios" es una suposición razonable para períodos sin datos.

In [ ]:
data.bfill()

,0
a,1
b,2
c,2
d,3
e,3


In [ ]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [ ]:
df.ffill(axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


## Ejercicio 1: Detección de Valores Nulos

```python
import pandas as pd
import numpy as np

# 1.1 Crea una Serie con los siguientes valores: [10, None, 20, np.nan, 30, pd.NA]
# Tu código aquí

# 1.2 Utiliza el método apropiado para identificar qué posiciones contienen valores nulos
# Tu código aquí

# 1.3 Calcula qué porcentaje de los valores en la Serie son nulos
# Tu código aquí

# 1.4 Crea un DataFrame con la siguiente información:
# - Columnas: 'A', 'B', 'C'
# - Datos:
#   * Fila 1: [1, np.nan, 3]
#   * Fila 2: [4, 5, np.nan]
#   * Fila 3: [np.nan, 8, 9]
#   * Fila 4: [10, None, 12]
# Tu código aquí

# 1.5 Identifica qué columnas contienen al menos un valor nulo
# Tu código aquí

# 1.6 Cuenta el número de valores nulos por columna
# Tu código aquí

# 1.7 Cuenta el número de valores nulos por fila
# Tu código aquí
```

## Ejercicio 2: Eliminación de Valores Nulos

```python
# Utilizando el DataFrame del ejercicio 1.4

# 2.1 Elimina todas las filas que contengan al menos un valor nulo
# Tu código aquí

# 2.2 Elimina solo las filas donde todos los valores son nulos
# (Primero agrega una fila de solo valores nulos al DataFrame)
# Tu código aquí

# 2.3 Elimina las columnas que tengan al menos un valor nulo
# Tu código aquí

# 2.4 Elimina solo las filas donde la columna 'A' tenga valores nulos
# Tu código aquí

# 2.5 Elimina las filas que tengan menos de 2 valores no nulos
# Tu código aquí
```

## Ejercicio 3: Rellenando Valores Nulos

```python
# 3.1 Crea una Serie de tiempo con fechas diarias para una semana y valores
# [10, np.nan, 15, np.nan, np.nan, 20, 25]
# Tu código aquí

# 3.2 Rellena todos los valores nulos con cero
# Tu código aquí

# 3.3 Rellena los valores nulos con la media de los valores no nulos
# Tu código aquí

# 3.4 Utiliza el método forward fill para propagar el último valor válido
# Tu código aquí

# 3.5 Utiliza el método backward fill para propagar el siguiente valor válido
# Tu código aquí

# 3.6 Utiliza una combinación de forward fill y backward fill para asegurarte
# de que no queden valores nulos (primero ffill, luego bfill)
# Tu código aquí

# 3.7 Crea un DataFrame con datos de ventas mensuales para 3 productos
# con algunos valores faltantes
# Tu código aquí

# 3.8 Rellena los valores nulos de cada producto con la media de ventas de ese producto
# Tu código aquí

# 3.9 Rellena los valores nulos con el valor de la misma columna de la fila anterior
# Tu código aquí
```

## Ejercicio 4: Tipos de Datos Anulables (Nullable Dtypes)

```python
# 4.1 Crea una Serie de enteros que incluya valores nulos,
# utilizando el tipo Int64 de Pandas
# Tu código aquí

# 4.2 Demuestra qué sucede si intentas convertir esta Serie a un tipo entero
# estándar de NumPy (np.int64)
# Tu código aquí

# 4.3 Crea un DataFrame con una columna de enteros, una de flotantes y una de cadenas,
# todas con algunos valores nulos, utilizando tipos anulables
# Tu código aquí

# 4.4 Realiza algunas operaciones aritméticas con la Serie de enteros anulables y
# observa cómo se manejan los valores NA
# Tu código aquí

# 4.5 Convierte una Serie regular con valores nulos a un tipo anulable apropiado
# Tu código aquí
```

In [ ]:
import pandas as pd
import numpy as np

# 1.1 Crea una Serie con los siguientes valores: [10, None, 20, np.nan, 30, pd.NA]
serie = pd.Series([10, None, 20, np.nan, 30, pd.NA])
print("Serie original:")
print(serie)

# 1.2 Utiliza el método apropiado para identificar qué posiciones contienen valores nulos
mascara_nulos = serie.isnull()
print("\nMáscara de valores nulos:")
print(mascara_nulos)

# 1.3 Calcula qué porcentaje de los valores en la Serie son nulos
porcentaje_nulos = mascara_nulos.mean() * 100
print(f"\nPorcentaje de valores nulos: {porcentaje_nulos:.1f}%")

# 1.4 Crea un DataFrame con la información especificada
df = pd.DataFrame({
    'A': [1, 4, np.nan, 10],
    'B': [np.nan, 5, 8, None],
    'C': [3, np.nan, 9, 12]
})
print("\nDataFrame:")
print(df)

# 1.5 Identifica qué columnas contienen al menos un valor nulo
columnas_con_nulos = df.columns[df.isna().any()].tolist()
print(f"\nColumnas con al menos un valor nulo: {columnas_con_nulos}")

# 1.6 Cuenta el número de valores nulos por columna
nulos_por_columna = df.isna().sum()
print("\nNúmero de valores nulos por columna:")
print(nulos_por_columna)

# 1.7 Cuenta el número de valores nulos por fila
nulos_por_fila = df.isna().sum(axis=1)
print("\nNúmero de valores nulos por fila:")
print(nulos_por_fila)

Serie original:
0      10
1    None
2      20
3     NaN
4      30
5    <NA>
dtype: object

Máscara de valores nulos:
0    False
1     True
2    False
3     True
4    False
5     True
dtype: bool

Porcentaje de valores nulos: 50.0%

DataFrame:
      A    B     C
0   1.0  NaN   3.0
1   4.0  5.0   NaN
2   NaN  8.0   9.0
3  10.0  NaN  12.0

Columnas con al menos un valor nulo: ['A', 'B', 'C']

Número de valores nulos por columna:
A    1
B    2
C    1
dtype: int64

Número de valores nulos por fila:
0    1
1    1
2    1
3    1
dtype: int64


In [ ]:
# Utilizando el DataFrame del ejercicio 1.4
print("DataFrame original:")
print(df)

# 2.1 Elimina todas las filas que contengan al menos un valor nulo
df_sin_nulos = df.dropna()
print("\nDataFrame sin filas con valores nulos:")
print(df_sin_nulos)

# 2.2 Elimina solo las filas donde todos los valores son nulos
# (Primero agrega una fila de solo valores nulos al DataFrame)
df2 = df.copy()
df2.loc[4] = [np.nan, np.nan, np.nan]
print("\nDataFrame con una fila adicional de todos nulos:")
print(df2)

df2_sin_filas_todos_nulos = df2.dropna(how='all')
print("\nDataFrame sin filas donde todos son nulos:")
print(df2_sin_filas_todos_nulos)

# 2.3 Elimina las columnas que tengan al menos un valor nulo
df_sin_columnas_nulas = df.dropna(axis=1)
print("\nDataFrame sin columnas con valores nulos:")
print(df_sin_columnas_nulas)

# 2.4 Elimina solo las filas donde la columna 'A' tenga valores nulos
df_a_no_nulo = df[df['A'].notna()]
print("\nDataFrame sin filas donde A es nulo:")
print(df_a_no_nulo)

# 2.5 Elimina las filas que tengan menos de 2 valores no nulos
df_al_menos_2_no_nulos = df.dropna(thresh=2)
print("\nDataFrame con filas que tienen al menos 2 valores no nulos:")
print(df_al_menos_2_no_nulos)

DataFrame original:
      A    B     C
0   1.0  NaN   3.0
1   4.0  5.0   NaN
2   NaN  8.0   9.0
3  10.0  NaN  12.0

DataFrame sin filas con valores nulos:
Empty DataFrame
Columns: [A, B, C]
Index: []

DataFrame con una fila adicional de todos nulos:
      A    B     C
0   1.0  NaN   3.0
1   4.0  5.0   NaN
2   NaN  8.0   9.0
3  10.0  NaN  12.0
4   NaN  NaN   NaN

DataFrame sin filas donde todos son nulos:
      A    B     C
0   1.0  NaN   3.0
1   4.0  5.0   NaN
2   NaN  8.0   9.0
3  10.0  NaN  12.0

DataFrame sin columnas con valores nulos:
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3]

DataFrame sin filas donde A es nulo:
      A    B     C
0   1.0  NaN   3.0
1   4.0  5.0   NaN
3  10.0  NaN  12.0

DataFrame con filas que tienen al menos 2 valores no nulos:
      A    B     C
0   1.0  NaN   3.0
1   4.0  5.0   NaN
2   NaN  8.0   9.0
3  10.0  NaN  12.0


In [ ]:
# 3.1 Crea una Serie de tiempo con fechas diarias para una semana
fechas = pd.date_range('2023-01-01', periods=7, freq='D')
serie_tiempo = pd.Series([10, np.nan, 15, np.nan, np.nan, 20, 25], index=fechas)
print("Serie de tiempo original:")
print(serie_tiempo)

# 3.2 Rellena todos los valores nulos con cero
serie_con_ceros = serie_tiempo.fillna(0)
print("\nSerie con nulos reemplazados por ceros:")
print(serie_con_ceros)

# 3.3 Rellena los valores nulos con la media de los valores no nulos
media = serie_tiempo.mean()
serie_con_media = serie_tiempo.fillna(media)
print(f"\nSerie con nulos reemplazados por la media ({media}):")
print(serie_con_media)

# 3.4 Utiliza el método forward fill
serie_ffill = serie_tiempo.fillna(method='ffill')
print("\nSerie con forward fill:")
print(serie_ffill)

# 3.5 Utiliza el método backward fill
serie_bfill = serie_tiempo.fillna(method='bfill')
print("\nSerie con backward fill:")
print(serie_bfill)

# 3.6 Combinación de forward fill y backward fill
serie_combinada = serie_tiempo.fillna(method='ffill').fillna(method='bfill')
print("\nSerie con ffill seguido de bfill:")
print(serie_combinada)

# 3.7 Crea un DataFrame con datos de ventas mensuales
meses = pd.date_range('2023-01-01', periods=6, freq='M')
ventas = pd.DataFrame({
    'Producto A': [100, 120, np.nan, 140, np.nan, 160],
    'Producto B': [200, np.nan, 220, np.nan, 240, 250],
    'Producto C': [np.nan, 300, 320, 340, 360, np.nan]
}, index=meses)
print("\nDataFrame de ventas mensuales:")
print(ventas)

# 3.8 Rellena los valores nulos con la media de cada producto
ventas_con_media = ventas.fillna(ventas.mean())
print("\nVentas con nulos reemplazados por la media de cada producto:")
print(ventas_con_media)

# 3.9 Rellena los valores nulos con el valor de la misma columna de la fila anterior
ventas_ffill = ventas.fillna(method='ffill')
print("\nVentas con forward fill por columna:")
print(ventas_ffill)

Serie de tiempo original:
2023-01-01    10.0
2023-01-02     NaN
2023-01-03    15.0
2023-01-04     NaN
2023-01-05     NaN
2023-01-06    20.0
2023-01-07    25.0
Freq: D, dtype: float64

Serie con nulos reemplazados por ceros:
2023-01-01    10.0
2023-01-02     0.0
2023-01-03    15.0
2023-01-04     0.0
2023-01-05     0.0
2023-01-06    20.0
2023-01-07    25.0
Freq: D, dtype: float64

Serie con nulos reemplazados por la media (17.5):
2023-01-01    10.0
2023-01-02    17.5
2023-01-03    15.0
2023-01-04    17.5
2023-01-05    17.5
2023-01-06    20.0
2023-01-07    25.0
Freq: D, dtype: float64

Serie con forward fill:
2023-01-01    10.0
2023-01-02    10.0
2023-01-03    15.0
2023-01-04    15.0
2023-01-05    15.0
2023-01-06    20.0
2023-01-07    25.0
Freq: D, dtype: float64

Serie con backward fill:
2023-01-01    10.0
2023-01-02    15.0
2023-01-03    15.0
2023-01-04    20.0
2023-01-05    20.0
2023-01-06    20.0
2023-01-07    25.0
Freq: D, dtype: float64

Serie con ffill seguido de bfill:
2023-01-01 

<ipython-input-47-3b2eda262163>:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  serie_ffill = serie_tiempo.fillna(method='ffill')
<ipython-input-47-3b2eda262163>:24: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  serie_bfill = serie_tiempo.fillna(method='bfill')
<ipython-input-47-3b2eda262163>:29: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  serie_combinada = serie_tiempo.fillna(method='ffill').fillna(method='bfill')
<ipython-input-47-3b2eda262163>:34: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  meses = pd.date_range('2023-01-01', periods=6, freq='M')
<ipython-input-47-3b2eda262163>:49: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a 

In [ ]:
# 4.1 Crea una Serie de enteros que incluya valores nulos
serie_int = pd.Series([1, 2, None, 4, np.nan, 6], dtype='Int64')
print("Serie con tipo Int64:")
print(serie_int)
print("Tipo de datos:", serie_int.dtype)

# 4.2 Demuestra qué sucede si intentas convertir esta Serie a np.int64
try:
    serie_numpy_int = serie_int.astype(np.int64)
    print("\nConversión exitosa (esto no debería verse):", serie_numpy_int)
except Exception as e:
    print(f"\nError al convertir a np.int64: {e}")

# 4.3 Crea un DataFrame con tipos anulables
df_nullable = pd.DataFrame({
    'enteros': pd.Series([1, 2, None, 4], dtype='Int64'),
    'flotantes': pd.Series([1.1, np.nan, 3.3, 4.4], dtype='Float64'),
    'cadenas': pd.Series(['a', None, 'c', 'd'], dtype='string')
})
print("\nDataFrame con tipos anulables:")
print(df_nullable)
print("\nTipos de datos:")
print(df_nullable.dtypes)

# 4.4 Operaciones aritméticas con la Serie de enteros anulables
print("\nSerie original:")
print(serie_int)
print("\nSuma de 10:")
print(serie_int + 10)
print("\nMultiplicación por 2:")
print(serie_int * 2)
print("\nSuma de series con valores nulos:")
print(serie_int + pd.Series([10, None, 30, 40, 50, 60], dtype='Int64'))

# 4.5 Convierte una Serie regular con valores nulos a un tipo anulable
serie_regular = pd.Series([1, 2, None, 4, np.nan])
print("\nSerie regular:")
print(serie_regular)
print("Tipo:", serie_regular.dtype)

serie_convertida = serie_regular.astype('Int64')
print("\nSerie convertida a Int64:")
print(serie_convertida)
print("Nuevo tipo:", serie_convertida.dtype)

Serie con tipo Int64:
0       1
1       2
2    <NA>
3       4
4    <NA>
5       6
dtype: Int64
Tipo de datos: Int64

Error al convertir a np.int64: cannot convert NA to integer

DataFrame con tipos anulables:
   enteros  flotantes cadenas
0        1        1.1       a
1        2       <NA>    <NA>
2     <NA>        3.3       c
3        4        4.4       d

Tipos de datos:
enteros               Int64
flotantes           Float64
cadenas      string[python]
dtype: object

Serie original:
0       1
1       2
2    <NA>
3       4
4    <NA>
5       6
dtype: Int64

Suma de 10:
0      11
1      12
2    <NA>
3      14
4    <NA>
5      16
dtype: Int64

Multiplicación por 2:
0       2
1       4
2    <NA>
3       8
4    <NA>
5      12
dtype: Int64

Suma de series con valores nulos:
0      11
1    <NA>
2    <NA>
3      44
4    <NA>
5      66
dtype: Int64

Serie regular:
0    1.0
1    2.0
2    NaN
3    4.0
4    NaN
dtype: float64
Tipo: float64

Serie convertida a Int64:
0       1
1       2
2    <NA>